**Step 1: Install Required Libraries**

In [1]:
!pip install -qU langchain langchain-community faiss-cpu pypdf sentence-transformers gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.

**Step 2: Imported Libraries**

In [2]:
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.5 MB/s eta 0:00:00


In [3]:
import os
import faiss
import numpy as np
import gradio as gr
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


**Step 3: Set Up API Key**

In this Project i use Groq api key

In [4]:
import os

os.environ["GROQ_API_KEY"] = "Paste_your_key"

**Step 4: Initialize LLM and Embedding Model**

* Use LLaMA3 model for better analysis
* Use HuggingFace Embeddings model

In [5]:
llm = ChatGroq(model="llama3-8b-8192")
embedding_model = HuggingFaceEmbeddings()

<ipython-input-5-3f52f9f75f29>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings()
<ipython-input-5-3f52f9f75f29>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_model = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https:

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Step 5 : Function to Load and Process PDF Files**

* Used Pypdf for extracting data from pdf files
* RecursiveCharacterTextSplitter used to divide text into small chunks

In [6]:
def process_pdf(file_path):
    """Extracts text from PDF and returns combined text."""
    loader = PyPDFLoader(file_path)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    split_docs = text_splitter.split_documents(docs)

    return " ".join([doc.page_content for doc in split_docs])

**Step 6 : Function to Embed Text and Store in FAISS**

* FAISS uses L2 distance to compare vectors and find the most similar ones.
* The smaller the L2 distance, the more similar the resume is to the job description.

In [7]:
def create_faiss_index(texts):
    """Creates FAISS index for storing resume embeddings."""
    vectors = embedding_model.embed_documents(texts)  # Convert text to embeddings
    dimension = len(vectors[0])  # Get embedding size

    faiss_index = faiss.IndexFlatL2(dimension)  # L2 Distance Index to find similar resume
    faiss_index.add(np.array(vectors))  # Add embeddings to FAISS

    return faiss_index, vectors  # Return FAISS index and vector embeddings

**Step 7 : Function to Compare Resumes with Job Description Using FAISS**

Query LLM for Detailed Analysis
* Extract text from Job description and resume
* Embeded and stored in vector db
* k is to find top matched results

In [8]:
def compare_resumes(job_desc_path, resume_paths):
    """Compares resumes with job description and returns results."""
    job_desc_text = process_pdf(job_desc_path)  # Extract job description text
    resume_texts = [process_pdf(resume) for resume in resume_paths]  # Extract resumes text

    # Create FAISS index for resumes
    faiss_index, resume_vectors = create_faiss_index(resume_texts)

    # Embed job description and perform FAISS search
    job_desc_vector = embedding_model.embed_documents([job_desc_text])[0]  # Get embedding for job description
    _, matched_indices = faiss_index.search(np.array([job_desc_vector]), k=len(resume_paths))  # Find best matches

    results = []  # Store results

    for rank, idx in enumerate(matched_indices[0]):  # Iterate over matched resumes
        resume_text = resume_texts[idx]
        resume_name = os.path.basename(resume_paths[idx])

        # Compute Similarity Score (Cosine Similarity)
        similarity = np.dot(job_desc_vector, resume_vectors[idx]) / (
            np.linalg.norm(job_desc_vector) * np.linalg.norm(resume_vectors[idx])
        )
        match_percentage = round(similarity * 100, 2)  # Convert to percentage

        # Query LLM for Detailed Analysis
        response = llm.invoke([
            HumanMessage(content=f"""
            Compare the following resume with the job description.
            Job Description: {job_desc_text}
            Resume: {resume_text}

            Provide the following details:
            1. **generate response in paragraph**
            2. **Key Skills Matched**
            3. **Missing Skills**
            4. **Suggestions for Improvement**

            Format the response in a structured manner.
            """)
        ])

        results.append(f"**🔹 Result for {resume_name}**\n\n**Match Percentage:** {match_percentage}%\n\n" + response.content)

    return "\n\n".join(results)


**Step 8 : Interface Gradio**

In [9]:
def resume_match_interface(job_desc_file, resume_files):
    """Handles file inputs from Gradio and returns match results."""
    if not job_desc_file or not resume_files:
        return "**❌ Please upload a job description and at least one resume.**"

    # Convert Gradio file objects to actual file paths
    job_desc_path = job_desc_file.name
    resume_paths = [resume.name for resume in resume_files]

    return compare_resumes(job_desc_path, resume_paths)  # Call main function


In [10]:
import time

In [11]:
with gr.Blocks() as demo:
    gr.Markdown("# 📄 Resume Matching AI Chatbot 🤖")

    with gr.Row():
        job_desc_input = gr.File(label="📄 Upload Job Description (PDF)", type="filepath")
        resume_inputs = gr.File(label="📄 Upload Resumes (PDF)", type="filepath", file_count="multiple")

    output_box = gr.Markdown()

    submit_button = gr.Button("🔍 Analyze Resumes")

    submit_button.click(resume_match_interface, inputs=[job_desc_input, resume_inputs], outputs=output_box)


def analyze_resume_with_loading(resumes, job_description):
    yield "🔄 Processing resumes... Please wait..."
    time.sleep(3)  # Simulating processing time
    yield "✅ Analysis Complete! Here are the results..."

# Step 🔟: Launch Gradio App
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://80dc41acd5ed61c8a7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
